In [1]:
import os
import json
import numpy as np
from pandas import DataFrame, read_csv
from typing import List
from transformers import BertModel, BertTokenizer, BertTokenizerFast
import torch
import torch.nn.functional as F



2023-03-02 20:51:23.832494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 20:51:23.932991: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 20:51:23.935793: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anaconda3/lib/:/home/ikonkobo/anacond

In [2]:
os.chdir("../")
from src.data.request_data import getDataTF, getDataHF
from src.data.processing_data import StackedFormat

In [3]:
with open(r"./inputs_data/tokens.json") as file:
    input = json.load(file)

model_id = input["model_id"]
hf_token = input["hf_token"]

In [4]:
# url_dyda_da = input["url_dyda_da"]
# df_dyda_da = getDataHF(url_dyda_da)
# getDataTF("dyda_da", "train")

In [5]:
Contexts, Labels = StackedFormat("dyda_da", "train", 5).get_contexts_labels()

In [6]:
Contexts[:2]

["that's true , but then you have to buy a record player . can you study with the radio on ? no , i listen to background music . what is the difference ? the radio has too many comerials .",
 "wow ! you're really working up a storm ! i know . i've even worked up a sweat . you look like a cooking show host--only messier . anyone home ? jen ! i'm in the kitchen ... let yourself in !"]

In [7]:
Labels

tensor([[2., 3., 2., 3., 2.],
        [2., 2., 2., 3., 2.],
        [2., 1., 2., 0., 3.],
        ...,
        [1., 3., 2., 2., 0.],
        [2., 2., 3., 2., 3.],
        [0., 1., 3., 3., 2.]])

## Testing BERT embedding

In [8]:
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
"""
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

input_ids_tensor = torch.tensor(tokenizer.batch_encode_plus(Contexts,
                                     max_length=50,
                                     padding=True,
                                     truncation="only_first")["input_ids"][:3])

with torch.no_grad():
    embedds = model(input_ids_tensor)[0]
embedds
"""


'\ntokenizer = BertTokenizerFast.from_pretrained(\'bert-base-uncased\')\n\ninput_ids_tensor = torch.tensor(tokenizer.batch_encode_plus(Contexts,\n                                     max_length=50,\n                                     padding=True,\n                                     truncation="only_first")["input_ids"][:3])\n\nwith torch.no_grad():\n    embedds = model(input_ids_tensor)[0]\nembedds\n'

In [10]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = Contexts[:10]

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 0.0083,  0.0223,  0.0323,  ...,  0.0124,  0.0278,  0.0150],
        [ 0.0267, -0.0055,  0.0405,  ...,  0.0071,  0.0359,  0.0112],
        [ 0.0153,  0.0066,  0.0517,  ..., -0.0032,  0.0127,  0.0094],
        ...,
        [ 0.0113, -0.0126,  0.0788,  ...,  0.0023,  0.0367, -0.0040],
        [ 0.0142,  0.0099,  0.0375,  ...,  0.0171,  0.0337,  0.0328],
        [ 0.0172, -0.0161,  0.0239,  ..., -0.0110,  0.0151,  0.0002]])


In [11]:
sentence_embeddings.shape

torch.Size([10, 768])

In [12]:
# model = BertModel.from_pretrained("bert-base-uncased")
# embedding_matrix = model.embeddings.word_embeddings.weight

In [13]:
#def embedding(texts: List[str],
#              model_id: str,
#              hf_token: str):
#    api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
#    headers = {"Authorization": f"Bearer {hf_token}"}
#    response = requests.post(api_url, 
#                             headers=headers, 
#                             json={"inputs": texts, 
#                                   "options":{"wait_for_model":True}})
#    return response.json()